"""
AI Agent Automation with LangGraph - Complete Implementation
Customer Support Automation System using Groq API
"""

In [1]:
import os
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage
import operator
from datetime import datetime
import json
import logging

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)

logger = logging.getLogger(__name__)

In [4]:
# ============================================================================
# STATE DEFINITION
# ============================================================================

class AgentState(TypedDict):
    query: str
    category: str
    retrieved_data: str
    response: str
    quality_score: int
    timestamp: str
    metadata: dict

In [5]:
# ============================================================================
# CONFIGURATION
# ============================================================================
class Config:
    """Configuration for AI Agent System"""
    GROQ_API_KEY = os.getenv("GROQ_API_KEY")
    MODEL_NAME = "llama-3.3-70b-versatile"
    TEMPERATURE = 0.7
    MAX_TOKENS = 1000
    
    #Knowledge Base Simulation
    KNOWLEDGE_BASE = {
        'ORDER_STATUS': {
            "data": "Order tracking system connected. Typical delivery: 3-5 business days.",
            "context": "Customer inquiring about order status"
        },
        'REFUND': {
            "data": "Refund policy: 30-day return window. Full refund with receipt. Processing time: 5-7 business days.",
            "context": "Customer requesting refund information"
        },
        'SHIPPING': {
            "data": "Shipping options: Standard (5-7 days, $5.99), Express (2-3 days, $12.99), Overnight ($24.99).",
            "context": "Customer inquiring about shipping"
        },
        'POLICY': {
            "data": "Return policy: 30 days, original condition. Warranty: 1 year manufacturer warranty on electronics.",
            "context": "Customer asking about policies"
        },
        'TECHNICAL_SUPPORT': {
            "data": "Technical support available 24/7. Common issues: password reset, account access, product setup.",
            "context": "Customer needs technical assistance"
        },
        'GENERAL': {
            "data": "General customer support. Business hours: Mon-Fri 9AM-6PM EST. Contact: support@company.com",
            "context": "General inquiry"
        }
    }

In [6]:
# ============================================================================
# LLM INITIALIZATION
# ============================================================================
def initialize_llm():
    """Initialize the Groq LLM"""
    try:
        llm = ChatGroq(
            model=Config.MODEL_NAME,
            temperature=Config.TEMPERATURE,
            max_tokens=Config.MAX_TOKENS,
            groq_api_key=Config.GROQ_API_KEY
        )
        logger.info("LLM initialized successfully")
        return llm
    except Exception as e:
        logger.error(f"Failed to initialized LLM: {e}")
        raise

In [7]:
# ============================================================================
# AGENT NODES
# ============================================================================
def classify_query_node(state: AgentState) -> AgentState:
    """
    Node 1: Classify the incoming query into categories
    """
    logger.info(f"Classifying query: {state['query']}")
    
    try:
        llm = initialize_llm()
        
        prompt = f"""Analyze this customer support query and classify it into ONE of this categories:
        - ORDER_STATUS: Questions about order tracking, delivery status
        - REFUND: Refund requests, return inquiries
        - SHIPPING: Shipping methods, costs, delivery times
        - POLICY: Return policies, warranties, terms
        - TECHNICAL_SUPPORT: Technical issues, troubleshooting
        - GENERAL: Everything else
        
        Query: {state['query']}
        
        Responde with ONLY the category name, nothing else."""
        
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        category = response.content.strip()
        
        logger.info(f"Query classified as: {category}")
        
        return {
            **state,
            "category": category,
            "timestamp": datetime.now().isoformat(),
            "metadata": {
                "node": "classifier",
                "status": "completed"
            }
        }
    except Exception as e:
        logger.error(f"Error in classify_query_node: {e}")
        return {
            **state,
            "category": "GENERAL",
            "metadata": {
                "node": "classifier",
                "status": "error",
                "error": str(e)
            }
        }

In [8]:
def retrieve_data_node(state: AgentState) -> AgentState:
    """
    Node 2: Retrieve relevant data based on classification
    """
    logger.info(f"Retrieving data for category: {state['category']}")
    
    try:
        category = state.get('category', "GENERAL")
        
        # Retrieve from knowledge base
        knowledge = Config.KNOWLEDGE_BASE.get(category, Config.KNOWLEDGE_BASE["GENERAL"])
        retrieved_data = knowledge["data"]
        
        logger.info(f"Data retrieved successfully for {category}")
        
        # Return updated state
        return {
            **state, # Unpack the previous state
            "retrieved_data": retrieved_data,
            "metadata": {
                **state.get("metadata", {}),
                "retriever": "completed",
                "data_source": "knowledge_base"
            }
        }
    except Exception as e:
        logger.error(f"Error retrieved_data_node: {e}")
        return {
            **state, 
            "retrieved_data": "Error retrieving data",
            "metadata": {
                **state.get("metadata", {}),
                "retriever": "error"
            }
        }

In [9]:
def generate_response_node(state: AgentState) -> AgentState:
    """
    Node 3: Generate a helpful response using LLM
    """
    logger.info("Generating response")
    
    try:
        llm = initialize_llm()
        
        system_prompt = """You are a professional and friendly customer support agent. Your goal is to provide helpful, acurate and empathetic reponses to customer queries. Keep responses concise but complete. Be professional yet warm"""
        
        user_prompt = f"""Based on this information: {state['retrieved_data']}, Provide a helpful response to this customer query: {state['query']}. Make the response personal, professionl and actionable."""
        
        messages = [
            SystemMessage(content=system_prompt),
            HumanMessage(content=user_prompt)
        ]
        
        response = llm.invoke(messages)
        generated_response = response.content.strip()
        
        logger.info(f"Response generated successfully")
        
        return {
            **state,
            "response": generated_response,
            "metadata": {
                **state.get("metadata", {}),
                "generator": "completed"
            }
        }
    except Exception as e:
        logger.error(f"Error in generate_response_node: {e}")
        return {
            **state,
            "response": "I appologize, but I'm having trouble generating a response. Please try again.",
            "metadata": {
                **state.get("metadata", {}),
                "generator": "error"
            }
        }

In [10]:
def quality_check_node(state: AgentState) -> AgentState:
    """
    Node 4: Check the quality of the generated response.
    """
    logger.info("Performing quality check")
    
    try:
        llm = initialize_llm()
        
        prompt = f"""Evaluate this customer support response on a scale of 1-10:
        
        Original Query: {state['query']}
        Response: {state['response']}
        
        Consider:
        - Relevance to the query
        - Helpfulness and completeness
        - Professionalism and tone
        - Clarity and conciseness
        
        Respond with ONLY a number from 1-10, nothing else
        """
        
        messages = [HumanMessage(content=prompt)]
        response = llm.invoke(messages)
        
        try:
            quality_score = int(response.content.strip())
        except ValueError:
            quality_score = 7 # Default score if parsing fails
        
        logging.info(f"Quality score: {quality_score}/10")
        
        return {
            **state,
            "quality_score": quality_score,
            "metadata": {
                **state.get("metadata", {}),
                "quality_checker": "completed",
                "final_status": "success"
            }
        }
    except Exception as e:
        logger.error(f"Error in quality_check_node: {e}")
        return {
            **state,
            "quality_score": 0,
            "metadata": {
                **state.get("metadata", {}),
                "quality_checker": "error",
            }
        }

In [11]:
def should_regenerate(state: AgentState) -> Literal["regenerate", "end"]:
    """
    Conditional Edge: Decide if response needs regeneration.
    """
    quality_score = state.get("quality_score", 0)
    
    # If quality score is too low, regenerate (max 1 retry to avoid loops)
    retry_count = state.get("metadata", {}).get("retry_count", 0)
    
    if quality_score < 6 and retry_count < 1:
        logger.info(f"Quality score too low ({quality_score}/10), regenerating...")
        return "regenerate"
    return "end"

In [16]:
# ============================================================================
# GRAPH CONSTRUCTION
# ============================================================================
def create_agent_workflow():
    """
    Create and compile the Langgraph workflow.
    """
    logger.info("Creating agent workflow graph")
    
    #Initialize the graph
    workflow = StateGraph(AgentState)
    
    # Add Nodes
    workflow.add_node("classifier", classify_query_node)
    workflow.add_node("retriever", retrieve_data_node)
    workflow.add_node("generator", generate_response_node)
    workflow.add_node("quality_checker", quality_check_node)
    
    # Define the flow
    workflow.set_entry_point("classifier")
    workflow.add_edge("classifier", "retriever")
    workflow.add_edge("retriever", "generator")
    workflow.add_edge("generator", "quality_checker")
    
    # Conditional edge for regeneration
    workflow.add_conditional_edges(
        "quality_checker",
        should_regenerate,
        {
            "regenerate": "generator",
            "end": END
        }
    )
    
    #Compile the graph
    app = workflow.compile()
    
    logger.info("Workflow compiled successfully")
    return app

In [17]:
# ============================================================================
# MAIN EXECUTION
# ============================================================================
def run_agent(query: str):
    """
    Run the agent workflow with a given query
    """
    logger.info(f"Starting agent workflow for query: {query}")
    
    # Create the workflow
    app = create_agent_workflow()
    
    initial_state = {
        "query": query,
        "category": "",
        "retrieved_data": "",
        "response": "",
        "quality_score": 0,
        "timestamp": datetime.now().isoformat(),
        "metadata": {
            "retry_count": 0
        }
    }
    
    # Run the workflow
    try:
        final_state = app.invoke(initial_state)
        logger.info("Workflow completed successfully")
        return final_state
    except Exception as e:
        logger.error(f"Error running workflow: {e}")
        raise
    

In [18]:
def display_results(result: AgentState):
    """
    Display the results in a formatted way.
    """
    print("\n" + "="*80)
    print("AI Agent Automation Results")
    print("="*80)
    print(f"\nOriginal Query: {result['query']}")
    print(f"\nCategory: {result['category']}")
    print(f"\nRetrieved Data: {result['retrieved_data']}")
    print(f"\nGenerated Response: {result['response']}/10")
    print(f"\nQuality Score: {result['quality_score']}")
    print(f"\nTimestamp: {result['timestamp']}")
    print(f"\nMetadata: {json.dumps(result.get("metadata", {}), indent=2)}")
    print("\n" + "="*80 + "\n")

In [19]:
# ============================================================================
# EXAMPLE USAGE
# ============================================================================
if __name__ == "__main__":
    # Set your Groq API key
    # os.environ["GROQ_API_KEY"] = "your_api_key_here"
    
    # Check if API key is set
    if not Config.GROQ_API_KEY:
        print("❌ Error: GROQ_API_KEY not set!")
        print("Please set your Groq API key:")
        print("export GROQ_API_KEY='your_api_key_here'")
        exit(1)
        
    # Example queries to test
    test_queries = [
        "What's the status of my order #12345?",
        "I need to return a defective product and get a refund",
        "How long does standard shipping take?",
        "What is your return policy?",
        "I'm having trouble logging into my account"
    ]
    
    print("\n🤖 AI Agent Automation with LangGraph")
    print("Using Groq API with Llama 3.3 70B\n")
    
    for i, query in enumerate(test_queries, 1):
        print("\n" + "="*80)
        print(f"TEST CASE {i}/{len(test_queries)}")
        print("\n" + "="*80)
        
        try:
            result = run_agent(query)
            display_results(result)
        except Exception as e:
            logger.error(f"Error processing query: {e}")
        
        if i < len(test_queries):
            input("\nPress Enter to continue to the next test case...")
            
    print("\n🎉 All test cases completed!\n")
    print("\n To use with your own queries, call: run_agent('your_query_here')")

2025-11-25 11:17:04,272 - __main__ - INFO - Starting agent workflow for query: What's the status of my order #12345?
2025-11-25 11:17:04,272 - __main__ - INFO - Creating agent workflow graph
2025-11-25 11:17:04,287 - __main__ - INFO - Workflow compiled successfully
2025-11-25 11:17:04,343 - __main__ - INFO - Classifying query: What's the status of my order #12345?



🤖 AI Agent Automation with LangGraph
Using Groq API with Llama 3.3 70B


TEST CASE 1/5



2025-11-25 11:17:05,180 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:17:06,097 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-25 11:17:06,112 - __main__ - INFO - Query classified as: ORDER_STATUS
2025-11-25 11:17:06,112 - __main__ - INFO - Retrieving data for category: ORDER_STATUS
2025-11-25 11:17:06,112 - __main__ - INFO - Data retrieved successfully for ORDER_STATUS
2025-11-25 11:17:06,117 - __main__ - INFO - Generating response
2025-11-25 11:17:06,710 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:17:07,276 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-25 11:17:07,281 - __main__ - INFO - Response generated successfully
2025-11-25 11:17:07,281 - __main__ - INFO - Performing quality check
2025-11-25 11:17:07,943 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:17:08,095 - httpx - INFO - HTTP Request: POST https:


AI Agent Automation Results

Original Query: What's the status of my order #12345?

Category: ORDER_STATUS

Retrieved Data: Order tracking system connected. Typical delivery: 3-5 business days.

Generated Response: Hi there, thank you for reaching out about the status of your order #12345. I've checked on it for you, and our system shows that it's currently being processed for delivery. Since we've recently connected our order tracking system, you can expect to receive updates on the status of your order soon. Typically, our deliveries take 3-5 business days. If you'd like to track your order in real-time, I can provide you with tracking information as soon as it's available. Would you like me to send you an update with the tracking details as soon as possible?/10

Quality Score: 9

Timestamp: 2025-11-25T11:17:06.112142

Metadata: {
  "node": "classifier",
  "status": "completed",
  "retriever": "completed",
  "data_source": "knowledge_base",
  "generator": "completed",
  "quality_che

2025-11-25 11:18:55,037 - __main__ - INFO - Starting agent workflow for query: I need to return a defective product and get a refund
2025-11-25 11:18:55,037 - __main__ - INFO - Creating agent workflow graph
2025-11-25 11:18:55,062 - __main__ - INFO - Workflow compiled successfully
2025-11-25 11:18:55,062 - __main__ - INFO - Classifying query: I need to return a defective product and get a refund


...


AI Agent Automation Results

Original Query: What is your return policy?

Category: POLICY

Retrieved Data: Return policy: 30 days, original condition. Warranty: 1 year manufacturer warranty on electronics.

Generated Response: I'd be happy to help you with our return policy. If for any reason you're not satisfied with your purchase, you can return it within 30 days of receipt. To be eligible for a return, the item must be in its original condition, with all original packaging and accessories included. If you'd like to initiate a return, please don't hesitate to reach out to me directly and I'll guide you through the process. Would you like me to provide more details or answer any specific questions you may have about returns?/10

Quality Score: 9

Timestamp: 2025-11-25T11:19:32.387191

Metadata: {
  "node": "classifier",
  "status": "completed",
  "retriever": "completed",
  "data_source": "knowledge_base",
  "generator": "completed",
  "quality_checker": "completed",
  "final_status

2025-11-25 11:19:47,986 - __main__ - INFO - Starting agent workflow for query: I'm having trouble logging into my account
2025-11-25 11:19:47,986 - __main__ - INFO - Creating agent workflow graph
2025-11-25 11:19:48,018 - __main__ - INFO - Workflow compiled successfully
2025-11-25 11:19:48,021 - __main__ - INFO - Classifying query: I'm having trouble logging into my account



TEST CASE 5/5



2025-11-25 11:19:48,633 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:19:48,800 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-25 11:19:48,803 - __main__ - INFO - Query classified as: TECHNICAL_SUPPORT
2025-11-25 11:19:48,809 - __main__ - INFO - Retrieving data for category: TECHNICAL_SUPPORT
2025-11-25 11:19:48,810 - __main__ - INFO - Data retrieved successfully for TECHNICAL_SUPPORT
2025-11-25 11:19:48,812 - __main__ - INFO - Generating response
2025-11-25 11:19:49,402 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:19:49,834 - httpx - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-11-25 11:19:49,838 - __main__ - INFO - Response generated successfully
2025-11-25 11:19:49,841 - __main__ - INFO - Performing quality check
2025-11-25 11:19:50,416 - __main__ - INFO - LLM initialized successfully
2025-11-25 11:19:50,600 - httpx - INFO - HTTP Reque


AI Agent Automation Results

Original Query: I'm having trouble logging into my account

Category: TECHNICAL_SUPPORT

Retrieved Data: Technical support available 24/7. Common issues: password reset, account access, product setup.

Generated Response: I'm so sorry to hear that you're having trouble logging into your account. I'm here to help you get back up and running as quickly as possible. Can you please try resetting your password by clicking on the "Forgot Password" link on the login page? If that doesn't work, I'd be happy to assist you with troubleshooting or provide additional support to regain access to your account. Would you like me to walk you through the process or would you prefer I take a closer look into the issue?/10

Quality Score: 9

Timestamp: 2025-11-25T11:19:48.803305

Metadata: {
  "node": "classifier",
  "status": "completed",
  "retriever": "completed",
  "data_source": "knowledge_base",
  "generator": "completed",
  "quality_checker": "completed",
  "final_sta